In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict

In [2]:
def log_arguments(fn): # note that function as input
    def new_function(*args,**kwargs): # we've seen these arguments before
        print ('positional arguments:')
        print (args)
        print ('keyword arguments:')
        print (kwargs)
        return fn(*args,**kwargs) # return a function
    return new_function

In [23]:
class FrozenLake:
    '''
    A class written to solve Question 1 in HW2, ECE276C at UCSD
    For more on frozenlake, check out - https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py
    
    The action have the following meanings:
        LEFT = 0
        DOWN = 1
        RIGHT = 2
        UP = 3
    '''
    def __init__(self):
        '''
        Initialize the frozen-lake environment
        '''
        self.env = gym.make ("FrozenLake-v0")
        self.numStates = self.env.observation_space.n
        self.numActions = self.env.action_space.n
        self.dt = 1
        
        self.initial_state = self.env.reset() #reset env and return initial state
        print('Environment map:')
        self.printMap()

    def printMap(self):
        print(self.env.desc)
    
    def generateRollout(self, policy=None, maxT = 100, initial_state = 0):
        '''
        A rollout is a series of (state, action) pairs which goes on until time maxT or we reach a terminal state.
        A terminal state is reached when done = True in the following statement:
        
        >>> obs,r,done = env.step(action)
        '''
        assert isinstance(maxT, (int, float)), 'maxT needs to be int or float'
        assert isinstance(initial_state, int)
        assert initial_state in range(self.numStates)
        
        states = [initial_state]
        actions = []
        
        t = 0
        while(t < maxT):
            if policy == None:
                a = self.env.action_space.sample()
            else:
                a = policy(self, states[-1])
            # Take a step using action a
            next_state, r, done, info = env.step(a)
            # Save next state and the action that led to it
            states.append(next_state)
            actions.append(a)
            if done:
                break
            t = t + self.dt

        assert len(states) - len(actions) == 1, 'Number of actions should be 1 less than number of states'
        rollout = {
            'state': states,
            'action': actions,
        }
        
        return rollout
    
#     @log_arguments
    def TestPolicy(self, policy, num_trials=100, timeLimit = False, maxT=5000):
        '''
        Returns the average rate of successful episodes over 100 trials for a deterministic policy
        '''
        assert isinstance(policy, np.ndarray) and len(policy) == self.numStates
        assert isinstance(num_trials, int) and num_trials>0
        assert isinstance(timeLimit, bool)
        assert isinstance(maxT, int) and maxT>0

        
        success_count = 0
        for i in range(num_trials):
            t = 0
            state = int(self.env.reset()) #resetting state to initial position
            while(True):
                a = policy[state] #getting action from policy
                next_state, r, done, info = self.env.step(a) #taking a step using action
#                 print('next_state, r = ', next_state, r)
                # Check if we reached goal, i.e. check for success
                if (done and r == 1.0):
                    success_count += 1
                    break
                
                # Checking if we fell in a hole
                elif done:
                    break
                
                if timeLimit and t>maxT:
                    print('Max time exceeded. Breaking out of loop')
                    break
                state = next_state
                t += self.dt
        return success_count/num_trials

    def LearnModel(self, num_samples = 100000):
        '''
        Returns transition probabilities and reward function
        
        p(s'|a, s) is accessed by typing p[s][a][s']
        r(s,a,s') is accessed by typing r[s][a][s']
        '''
        assert isinstance(num_samples, int) and num_samples > 0
        
        self.env.reset()
        p = np.zeros((self.numStates, self.numActions, self.numStates))
        r = np.zeros((self.numStates, self.numActions, self.numStates))
        counter = np.zeros((self.numStates, self.numActions))
        self.count_s = np.zeros(self.numStates)
        self.count_a = np.zeros(self.numActions)
        
        for i in range(num_samples):
            s = np.random.randint(low = 0, high = self.numStates, dtype = int)
            a = np.random.randint(low = 0, high = self.numActions, dtype = int)
            self.count_s[s] += 1; self.count_a[a] += 1
            
            self.env.unwrapped.s = s #setting current state to randomly chosen state
            s_prime, reward, _, _ = self.env.step(a)
            
            p[s][a][s_prime] += 1
            r[s][a][s_prime] += reward
            counter[s][a] += 1
        
        #use itertools instead
        for s in range(self.numStates):
            for a in range(self.numActions):
                assert counter[s][a] != 0, 'Zero occurences of state-action pair. Cannot divide by 0'
                p[s][a][:] = p[s][a][:]/counter[s][a]
                r[s][a][:] = r[s][a][:]/counter[s][a]
        
        # Checking that probabilities sum to 1        
        for s in range(self.numStates):
            for a in range(self.numActions):
                assert abs(sum(p[s,a,:])-1.0) < 1e-4, 'Probabilities dont sum to 1 --> %f' % sum(p[s,a,:])
        
        return p, r, counter
    
    def initializeValueAndPolicyFunction(self, initial_value = 0):
        '''
        Initializes Value and Policy function and returns them
        All initial values for Value function are set to param initial_value
        Initial values for Policy function are chosen randomly between [0, number of actions)
        '''
        assert isinstance(initial_value, (int, float))
        
        V = np.zeros((self.numStates))
        policy = np.zeros((self.numStates))
        
        for s in range(self.numStates):
            a = np.random.randint(low = 0, high = self.numActions, dtype = int)
            V[s] = initial_value
            policy[s] = a
        
        return V, policy
    
    def evaluatePolicy(self, V, policy, gamma = 0.9):
        '''
        Evaluates policy for 1 iteration
        '''
        assert isinstance(V, np.ndarray) and len(V) == self.numStates
        assert isinstance(gamma, float) and 0.0<gamma<1.0
        
        V_new = np.zeros_like(V)
        for s in range(self.numStates):
            a = policy[s]
            pf_s = self.env.P[s][a] #prob distribution over states for taking action a at state s
            # Calculating expected value of Value function coz of policy
            exp_V = 0.0
            for possible_sa_pairs in pf_s:
                prob, next_state, r, _ = possible_sa_pairs
                exp_V += prob*( r + gamma*V[next_state] ) 
            V_new[s] = exp_V
        return V_new
                
    def checkPolicyConvergence(self, V_old, V_new, threshold = 1e-3):
        '''
        Checks for convergence by checking if the maximum difference between value of all states 
        is less than some threshold
        '''
        assert isinstance(threshold, float) and 0 < threshold < 1
        assert isinstance(V_old, np.ndarray) and len(V_old) == self.numStates
        assert isinstance(V_new, np.ndarray) and len(V_new) == self.numStates
        
        max_diff = np.max(np.abs(V_old - V_new))
        success = max_diff<threshold
        return success, max_diff

In [24]:
# def policyOne(fl, s):
#     '''
#     Returns action a by applying for the policy pi(s) = (s+1)%4 for state s
#     '''
#     assert isinstance(s, int)
#     assert s in range(fl.numStates)
    
#     a = (s+1)%4 #policy pi(s)
#     assert a in range(fl.numActions)
#     return a


In [25]:
fl = FrozenLake()
# Forming test policy
policyTest = np.zeros((fl.numStates))
for s in range(fl.numStates):
    policyTest[s] = (s+1)%4

success_rate = fl.TestPolicy(policyTest, num_trials = 100, maxT = 5000)
print('\nSuccess Rate for policy is: %.3f' % (success_rate))

Environment map:
[[b'S' b'F' b'F' b'F']
 [b'F' b'H' b'F' b'H']
 [b'F' b'F' b'F' b'H']
 [b'H' b'F' b'F' b'G']]

Success Rate for policy is: 0.000


In [26]:
p, r, counter = fl.LearnModel()

In [27]:
# 1. Initialize value and policy function
V, policy = fl.initializeValueAndPolicyFunction()

counter2 = 0
while True:
    # 2. Repeated policy Evaluation
    counter1 = 0
    while (True):
        # Step 1: Policy evaluation
        V_new = fl.evaluatePolicy(V, policy=policy, gamma = 0.9)
        success, max_diff = fl.checkPolicyConvergence(V, V_new, threshold = 1e-3)
        V = V_new
        if success:
#             print('Policy eval converged. max diff = ', max_diff)
            break
        else:
            counter1 += 1
    print('Policy Evaluation converged in %d iterations' % counter1)
    
    # 3. Policy improvement
    policy_stable = True
    gamma = 0.9

    for s in range(fl.numStates):
        old_action = policy[s]
        # Calculating q_pi(s,a) (from slides) for all actions over which we'll do argmax to find optimal action a
        q_pi_sa = np.zeros((fl.numActions))
        for a in range(fl.numActions):
            pf_s = fl.env.P[s][a] #prob distribution over states for taking action a at state s
            expectation = 0.0
            for possible_sa_pairs in pf_s:
                prob, next_state, r, _ = possible_sa_pairs
                expectation += prob*( r + gamma*V[next_state] ) 
            q_pi_sa[a] = expectation
        # Updating policy
        optimal_a = np.argmax(q_pi_sa)
        policy[s] = optimal_a 

        if old_action != optimal_a:
            policy_stable = False
    
    avg_success_rate = fl.TestPolicy(policy, num_trials = 100, maxT = 5000)
    print('avg_success_rate = ', avg_success_rate)
    if policy_stable:
        break;
    counter2 += 1
print('----->Policy improvement converged in %d iterations' % counter2)

Policy Evaluation converged in 9 iterations
avg_success_rate =  0.33
Policy Evaluation converged in 17 iterations
avg_success_rate =  0.38
Policy Evaluation converged in 8 iterations
avg_success_rate =  0.68
Policy Evaluation converged in 3 iterations
avg_success_rate =  0.71
----->Policy improvement converged in 3 iterations


In [28]:
print(V.reshape(4,4))

[[0.06302092 0.05648566 0.07094107 0.05232865]
 [0.086699   0.         0.11047763 0.        ]
 [0.14151271 0.24507633 0.29794938 0.        ]
 [0.         0.37831504 0.63821312 0.        ]]


In [29]:
print(policy.reshape(4,4))

[[0. 3. 0. 3.]
 [0. 0. 0. 0.]
 [3. 1. 0. 0.]
 [0. 2. 1. 0.]]


In [30]:
fl.TestPolicy(policy, num_trials = 100, maxT = 5000)

0.73

In [ ]:




## Plotting transition probabilites for a certain action a
# fig, axs = plt.subplots(figsize = (6,6))
# axs = plt.imshow(p[:,0,:], cmap='gray')
# plt.colorbar(axs,fraction=0.046, pad=0.04)
## Checking if states and actions were sampled uniformly
# fig, axs = plt.subplots(1,2, figsize = (15,5))
# axs[0].bar(range(fl.numStates), fl.count_s)
# axs[1].bar(range(fl.numActions), fl.count_a)

In [ ]:
# dir(gym.envs.toy_text.discrete.DiscreteEnv)
# import inspect
# inspect.getmembers(gym.envs.toy_text.discrete.DiscreteEnv, lambda a: not(inspect.isroutine(a)))